In [1]:
# import the necessary packages
import datetime
import matplotlib
import numpy as np
from keras.optimizers import SGD
from base_model_vgg import MiniVGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

/home/vishnu/.virtualenvs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def process_data(data, batch_size, label_binarizer):
    '''
    Function to read images from metadata
    process image for training, get label info
    from meta data
    '''
    
    with open(data, "r") as f:
        while True:
            images = []
            labels = []
            # process images in batches
            while len(images) < batch_size:
                line = f.readline()
                if line == "":
                    f.seek(0)
                    line = f.readline()

                line = line.strip().split(",")
                label = line[0]
                image = np.array([int(x) for x in line[1:]], dtype="uint8")
                image = image.reshape((64, 64, 3))
                # append features and target labels
                images.append(image)
                labels.append(label)

            labels = label_binarizer.transform(np.array(labels))
            # batches of tensor image data with real-time data augmentation
            data_aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                                          height_shift_range=0.2, shear_range=0.15, 
                                          horizontal_flip=True, fill_mode="nearest")

            (images, labels) = next(data_aug.flow(np.array(images),
                                                  labels, batch_size=batch_size))

            yield (np.array(images), labels)

In [3]:
# Setting up Variables, Hyper parameters
train_csv = "train.csv"
test_csv = "test.csv"

epoch_num = 200
size_batch = 16

num_train_images = 0
num_test_images = 0

In [4]:
# open meta data file
f = open(train_csv, "r")
labels = set()
testLabels = []

# loop over all rows of meta data
for line in f:
    # strip label values
    label = line.strip().split(",")[0]
    labels.add(label)
    num_train_images += 1
f.close()
f = open(test_csv, "r")

for line in f:
    label = line.strip().split(",")[0]
    testLabels.append(label)
    num_test_images += 1

# close the meta data file
f.close()

In [5]:
# binarize labels in a one-vs-all fashion

lb = LabelBinarizer()
lb.fit(list(labels))
testLabels = lb.transform(testLabels)

In [6]:
# initialize train and test image generators
train_obj = process_data(train_csv, size_batch, lb)
test_obj = process_data(test_csv, size_batch, lb)

In [7]:
# initialise MiniVGGNet
model = MiniVGGNet.build(64, 64, 3, len(lb.classes_))
opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / epoch_num)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# num_test_images

In [ ]:
# train the model
print("training/ generating model ...")
model.fit_generator(train_obj, steps_per_epoch=num_train_images // size_batch,
                    validation_data=test_obj, validation_steps=num_test_images // size_batch,
                    epochs=epoch_num)

training/ generating model ...
Epoch 1/200
620/620 [==============================] - 532s 858ms/step - loss: 8.4282 - acc: 0.0183 - val_loss: 5.1212 - val_acc: 0.0246
Epoch 2/200
620/620 [==============================] - 530s 855ms/step - loss: 7.8710 - acc: 0.0210 - val_loss: 4.9387 - val_acc: 0.0215
Epoch 3/200
620/620 [==============================] - 530s 855ms/step - loss: 7.4676 - acc: 0.0291 - val_loss: 4.7645 - val_acc: 0.0312
Epoch 4/200
620/620 [==============================] - 529s 853ms/step - loss: 7.1546 - acc: 0.0296 - val_loss: 4.9143 - val_acc: 0.0291
Epoch 5/200
620/620 [==============================] - 528s 852ms/step - loss: 6.8917 - acc: 0.0341 - val_loss: 4.5576 - val_acc: 0.0370
Epoch 6/200
620/620 [==============================] - 528s 852ms/step - loss: 6.6504 - acc: 0.0391 - val_loss: 4.4864 - val_acc: 0.0379
Epoch 7/200
620/620 [==============================] - 527s 850ms/step - loss: 6.4388 - acc: 0.0378 - val_loss: 4.5586 - val_acc: 0.0364
Epoch 8/20

In [ ]:
# re-initialize test data generator, for evaluating
test_obj = process_data(test_csv, size_batch, lb)

# make predictions on the testing images, finding the index of the
# label with the corresponding largest predicted probability
predIdxs = model.predict_generator(
    test_obj, steps=(num_test_images // size_batch) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
model.save('model_sample2.h5')